In [ ]:
# 런타임 GPU로 변경
# 구글 드라이브 마운트

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
# Flatten layer -> dropout -> Hidden layer (Denselayer)
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Raw Data Loading
df = pd.read_csv('/content/drive/My Drive/MachineLearning/train.csv')

# 결측치와 이상치는 없다.!!

# Data Split(Train data와 Test data 분리)
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False),
                 df['label'],
                 test_size=0.3,
                 random_state=0)

# Min-Max Normalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train, x_data_test

#### Tensorflow 2.x 구현 ### 

# 전체 box 생성 후 layer를 순차적으로 쌓는다.
model = Sequential()

# filter 몇개 사용할지 명시
model.add(Conv2D(filters=32,
                 kernel_size=(3,3),
                 activation='relu',
                 input_shape=(28, 28, 1)
                 ))
model.add(MaxPooling2D(pool_size=(2,2)))
# 인풋의 형태 명시 28, 28 사이즈 channel 1 (흑백) => 여러개지만 shape만 표기
# kernel size = stride size => 그래야 모든 사이즈 균등하게 pooling

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 activation='relu',
                 ))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,
                 kernel_size=(3,3),
                 activation='relu',
                 input_shape=(28, 28, 1)
                 ))

model.add(Flatten())

model.add(Dropout(rate=0.5)) 

# hidden layer는 dense로 표현, hidden layer 안에 256개의 로지스틱 배치
model.add(Dense(units=256,
                activation='relu'))

# 마지막은 10개의 카테고리
model.add(Dense(units=10,
                activation='softmax'))

print('===============model.sumaary()==================================')
print(model.summary())
print('================================================================')

# Back propagation으로 반복하여 w값 변경 시킬 때 사용할 optimizer => Adam
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
## 비용 함수 crossentropy 중, binary냐 categorical
## sparse -> one hot encoding 여부
## accuracy 같이 학습 -> metrics

# x_data_train_norm 2차원 (1차원 이미지 정보 여러장) => CNN은 4차원 입력값 필요
history = model.fit(x_data_train_norm.reshape(-1,28,28,1),
                    t_data_train,
                    epochs=100,
                    batch_size=100,
                    verbose=1,
                    validation_split=0.3)

# sparse_categorical_accuracy: 0.9981 => train data 정확도
# val_sparse_categorical_accuracy: 0.9915 => test data 정확도 => 꽤 높게 나옴

In [ ]:
model.evaluate(x_data_test_norm.reshape(-1,28,28,1),
               t_data_test)

In [ ]:
print(history.history.keys())
# dict_keys(['loss', 'sparse_categorical_accuracy', 'val_loss', 'val_sparse_categorical_accuracy'])
#            trianing data loss, trianing data accuracy, test data loss, test data accuracy
plt.plot(history.history['sparse_categorical_accuracy'],color='r')
plt.plot(history.history['val_sparse_categorical_accuracy'],color='b')
plt.ylim(0.95, 1.0) # y축에 대한 limit 설정
plt.plot() # 간격이 벌어지는 순간부터 overfitting이 시작됨